# All complexes prerare

In [ ]:
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB import PDBList, PDBIO, Select
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

In [ ]:
df_all_complexes = pd.read_excel('PPComplexes_pKD.xlsx', index_col=0, header=1)
df_all_complexes

,PDB code,Subset,Complex Type,Resolution,Affinity Data,pKd pKi pIC50,Release Year,Protein Name,Ligand Name,Pubmed ID,...,Exact Mass,No. of atoms,No. of bonds,Polar Surface Area,XLOGP3,open banel LogP,HB donor,HB acceptor,Rotatable bonds,Canonical SMILES
ID,,,,,,,,,,,,,,,,,,,,,
1,1a22,general,Protein-Protein,2.6,Kd=0.34nM,9.47,1998,GROWTH HORMONE RECEPTOR,G120R mutant human growth hormone (hGH),9571026.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5bxq,general,Protein-Protein,2.5,Kd=150nM,6.82,1998,Ran GTPase,Nuclear Transport Factor 2,9533885.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1a3b,general,Protein-Protein,1.8,Kd=0.3nM,9.52,1998,thrombin alpha,21-mer,8272424.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1a4y,general,Protein-Protein,2,Ki=1fM,15.00,1998,ANGIOGENIN,RIBONUCLEASE INHIBITOR,9311977.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1acb,general,Protein-Protein,2,Kd=0.2nM,9.70,1993,bovine alpha-chymotrypsin,leech (Hirudo medicinalis) protein proteinase ...,3071573.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,6um5,general,Protein-Protein,4.2,Kd=532nM,6.27,2019,CH848 10.17DT Env,HIV-1 neutralizing antibody DH270 UCA3,31806786.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2846,6umt,general,Protein-Protein,1.99,Kd=2.6nM,8.59,2019,"Programmed cell death protein 1, human PD-1 (N...","Programmed cell death 1 ligand 2, PD-L2 IgV",31727844.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2847,6uvo,general,Protein-Protein,2.9,Kd=579pM,9.24,2019,"RSV G central conserved domain, RSV G (157-197)",broadly neutralizing human monoclonal antibody...,31852779.0,...,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [ ]:
from math import sqrt
# Function for finding points with the greatest distance
def dist(p1, p2):
  x0 = p1[0] - p2[0]
  y0 = p1[1] - p2[1]
  z0 = p1[2] - p2[2]
  return x0 * x0 + y0 * y0 + z0 * z0


def maxDist(p):
  n = p.shape[0]
  maxm = 0
  for i in range(n):
    for j in range(i + 1, n):
      #print(dist(p.iloc[i], p.iloc[j]))
      if dist(p.iloc[i], p.iloc[j]) > maxm:
       maxm = dist(p.iloc[i], p.iloc[j])
       max_1 = i
       max_2 = j

  return [max_1, max_2, sqrt(maxm)]


In [ ]:
# Function for finding the intersection point of a vector with a plane
def add_v3v3(v0, v1):
    return (
        v0[0] + v1[0],
        v0[1] + v1[1],
        v0[2] + v1[2],
    )


def sub_v3v3(v0, v1):
    return (
        v0[0] - v1[0],
        v0[1] - v1[1],
        v0[2] - v1[2],
    )


def dot_v3v3(v0, v1):
    return (
        (v0[0] * v1[0]) +
        (v0[1] * v1[1]) +
        (v0[2] * v1[2])
    )


def len_squared_v3(v0):
    return dot_v3v3(v0, v0)


def mul_v3_fl(v0, f):
    return (
        v0[0] * f,
        v0[1] * f,
        v0[2] * f,
    )

def isect_line_plane_v3_4d(p0, p1, plane, epsilon=1e-6):
    u = sub_v3v3(p1, p0)
    dot = dot_v3v3(plane, u)

    if abs(dot) > epsilon:
        # Calculate a point on the plane
        # (divide can be omitted for unit hessian-normal form).
        p_co = mul_v3_fl(plane, -plane[3] / len_squared_v3(plane))

        w = sub_v3v3(p0, p_co)
        fac = -dot_v3v3(plane, w) / dot
        u = mul_v3_fl(u, fac)
        return add_v3v3(p0, u)

    return None

In [ ]:
from math import radians, sin, cos

def rotate_point_wrt_center(point_to_be_rotated, angle, center_point = (0,0)):

    angle = radians(angle)

    xnew = cos(angle)*(point_to_be_rotated[0] - center_point[0]) - sin(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[0]
    ynew = sin(angle)*(point_to_be_rotated[0] - center_point[0]) + cos(angle)*(point_to_be_rotated[1] - center_point[1]) + center_point[1]

    return [xnew, ynew]

rotate_point_wrt_center(point_to_be_rotated = (0,1), angle = 0, center_point = (0, 0)) # angle is negative to indicate clock-wise rotatio

[0.0, 1.0]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def plot_linear_cube(x, y, z, dx, dy, dz, color='red'):
    xx = [x, x, x+dx, x+dx, x]
    yy = [y, y+dy, y+dy, y, y]
    kwargs = {'alpha': 1, 'color': color}
    ax.plot3D(xx, yy, [z]*5, **kwargs)
    ax.plot3D(xx, yy, [z+dz]*5, **kwargs)
    ax.plot3D([x, x], [y, y], [z, z+dz], **kwargs)
    ax.plot3D([x, x], [y+dy, y+dy], [z, z+dz], **kwargs)
    ax.plot3D([x+dx, x+dx], [y+dy, y+dy], [z, z+dz], **kwargs)
    ax.plot3D([x+dx, x+dx], [y, y], [z, z+dz], **kwargs)

In [ ]:
import os
list_str = os.listdir('All_atoms_tables/Frames')

In [ ]:

for str_num in range(len(list_str)):

    struct_id = list_str[str_num][:-4]
    print(f'Structure {struct_id} number {str_num} in processing...')
    Atoms_coord_df = pd.read_csv(f'All_atoms_tables/{struct_id}.csv')
    if len(set(Atoms_coord_df['Chain_num'].values))<2:
        with open("Dataset/Diff_struct_10.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([str_num, struct_id])
        continue
    logreg_clf = LogisticRegression()
    logreg_clf.fit(Atoms_coord_df[['x', 'y', 'z']].values, Atoms_coord_df['Chain_num'].astype('int'))
    # plane_bind - coefficient list [A,B,C,D] in the separating plane equation of the form A*x+B*y+C*z+D=0
    plane_bind = list(logreg_clf.coef_[0])
    plane_bind.append(logreg_clf.intercept_[0])

    distant=10
    abs_d = distant*(np.sqrt(plane_bind[0]**2+plane_bind[1]**2+plane_bind[2]**2))
    coeff_up = -abs_d + plane_bind[3]
    coeff_down = abs_d + plane_bind[3]

    # limiting the area for searching for centers of mass
    plane_10_1=[plane_bind[0], plane_bind[1], plane_bind[2], coeff_down]
    plane_10_2=[plane_bind[0], plane_bind[1], plane_bind[2], coeff_up]
    # Obtaining centers of mass for proteins
    center_1=[]
    x1=0
    y1=0
    z1=0
    k1=0
    center_2=[]
    x2=0
    y2=0
    z2=0
    k2=0

    Atoms_3A_1 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
    Atoms_3A_2 = pd.DataFrame(columns=['Num', 'x', 'y', 'z', 'd'])
    Atoms_coord = pd.DataFrame(columns=['Num', 'x', 'y', 'z'])
    for j in range(Atoms_coord_df.shape[0]):
        # substitute the coordinates into the equation of the boundary planes, and select points inside the area limited by them
        d_down = plane_10_1[0]*Atoms_coord_df['x'][j] + plane_10_1[1]*Atoms_coord_df['y'][j] + plane_10_1[2]*Atoms_coord_df['z'][j] + plane_10_1[3]
        d_up = plane_10_2[0]*Atoms_coord_df['x'][j] + plane_10_2[1]*Atoms_coord_df['y'][j] + plane_10_2[2]*Atoms_coord_df['z'][j] + plane_10_2[3]
        if abs(d_down)<1:
            Atoms_3A_1.loc[len(Atoms_3A_1.index)] = [j, Atoms_coord_df['x'][j], Atoms_coord_df['y'][j], Atoms_coord_df['z'][j], d_down]
        if abs(d_up)<1:
            Atoms_3A_2.loc[len(Atoms_3A_2.index)] = [j,Atoms_coord_df['x'][j], Atoms_coord_df['y'][j], Atoms_coord_df['z'][j], d_up]
        if d_down >=0 and d_up <=0:
            Atoms_coord.loc[len(Atoms_coord.index)] = [j, Atoms_coord_df['x'][j], Atoms_coord_df['y'][j], Atoms_coord_df['z'][j]]
            if Atoms_coord_df['Chain_num'][j]==0:
                x1+=Atoms_coord_df['x'][j]
                y1+=Atoms_coord_df['y'][j]
                z1+=Atoms_coord_df['z'][j]
                k1+=1
            elif Atoms_coord_df['Chain_num'][j]==1:
                x2+=Atoms_coord_df['x'][j]
                y2+=Atoms_coord_df['y'][j]
                z2+=Atoms_coord_df['z'][j]
                k2+=1
    if k1 == 0 or k2 == 0:
        with open("Dataset/Diff_struct_10.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([str_num, struct_id])
        continue
    center_1=[round(x1/k1), round(y1/k1), round(z1/k1)]
    center_2=[round(x2/k2), round(y2/k2), round(z2/k2)]
    if Atoms_3A_2.shape[0]>=Atoms_3A_1.shape[0]:
        Atoms_3A = Atoms_3A_2
    else:
        Atoms_3A = Atoms_3A_1

    if Atoms_3A.shape[0] < 2:
        with open("Dataset/Diff_struct_10.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([str_num, struct_id])
        continue
    dots_max_dist = maxDist(Atoms_3A[['x', 'y', 'z']])

    #dot_is - coordinates of the intersection point, and therefore the cell center
    dot_is = isect_line_plane_v3_4d(center_1, center_2, plane_bind)
    x0, y0, z0 = dot_is[0], dot_is[1], dot_is[2]
    x1, y1, z1 = center_1[0], center_1[1], center_1[2]
    x2, y2, z2 = center_2[0], center_2[1], center_2[2]
    n_v = [(x1 - x0), (y1 - y0), (z1 - z0)]

    #vector connecting points with the greatest distance
    vec_3 = [[Atoms_3A['x'][dots_max_dist[0]] - Atoms_3A['x'][dots_max_dist[1]], Atoms_3A['y'][dots_max_dist[0]] - Atoms_3A['y'][dots_max_dist[1]], Atoms_3A['z'][dots_max_dist[0]] - Atoms_3A['z'][dots_max_dist[1]]]]

    # Obtaining the projection of the vector vec_5 onto the dividing plane
    # vector u
    u = np.array(vec_3)

    # vector n: n is orthogonal vector to Plane P
    n = np.array([plane_bind[0], plane_bind[1], plane_bind[2]])

    # Task: Project vector u on Plane P

    # finding norm of the vector n
    n_norm = np.sqrt(sum(n**2))

    # Apply the formula as mentioned above
    # for projecting a vector onto the orthogonal vector n
    # find dot product using np.dot()
    proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
    proj_vec = u - proj_of_u_on_n
    proj_vec = np.reshape(proj_vec, -1)

    norm = plane_bind[:3]
    a = sqrt(1/sum([coor**2 for coor in norm]))
    norm = [a*coor for coor in norm]
    norm = np.array(norm)
    a = sqrt(1/sum([coor**2 for coor in proj_vec]))
    proj_vec = a*proj_vec
    last_vector = np.cross(norm, proj_vec)

    Atoms_coord = Atoms_coord_df.copy()
    Atoms_coord['x'] = Atoms_coord['x']-dot_is[0]
    Atoms_coord['y'] = Atoms_coord['y']-dot_is[1]
    Atoms_coord['z'] = Atoms_coord['z']-dot_is[2]

    from_bazis = np.concatenate((np.expand_dims(proj_vec, axis=1),
                             np.expand_dims(last_vector, axis=1),
                             np.expand_dims(norm, axis=1)
                            ), axis=1)
    to_bazis = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
    s = np.matmul(np.linalg.inv(from_bazis), to_bazis)
    x_new = []
    y_new = []
    z_new = []
    for i, item in Atoms_coord.iterrows():
        vec = np.array([[item['x']], [item['y']], [item['z']]])
        new_vec = np.matmul(s, vec)
        new_xy = rotate_point_wrt_center(point_to_be_rotated = (new_vec[0][0], new_vec[1][0]), angle = 0, center_point = (0, 0))
        x_new.append(new_xy[0])
        y_new.append(new_xy[1])
        z_new.append(new_vec[2][0])
    Atoms_coord['x_new'] = x_new
    Atoms_coord['y_new'] = y_new
    Atoms_coord['z_new'] = z_new

    high_gr = 20
    length_gr = 40
    width_gr = 40
    high_v = [[0, 0, -high_gr], [0, 0, high_gr]]
    length_v = [[-length_gr, 0, 0], [length_gr, 0, 0]]
    width_v = [[0, -width_gr, 0], [0, width_gr, 0]]


    fig = plt.figure(figsize=(16, 10))
    ax = plt.axes(projection="3d")
    ax.plot3D([high_v[0][0],high_v[1][0]], [high_v[0][1],high_v[1][1]], [high_v[0][2],high_v[1][2]], 'red', linewidth= 5)
    ax.plot3D([length_v[0][0],length_v[1][0]], [length_v[0][1],length_v[1][1]], [length_v[0][2],length_v[1][2]], 'green', linewidth= 5)
    ax.plot3D([width_v[0][0],width_v[1][0]], [width_v[0][1],width_v[1][1]], [width_v[0][2],width_v[1][2]], 'blue', linewidth= 5)
    ax.scatter3D(Atoms_coord['x_new'], Atoms_coord['y_new'], Atoms_coord['z_new'], c = Atoms_coord['Chain_num'])
    plot_linear_cube(-length_gr, -width_gr, -high_gr, length_gr*2, width_gr*2, high_gr*2, color='gray')

    ax.view_init(3, 40)
    plt.savefig(f'Cell_images_10/{struct_id}.png')
    plt.close(fig)

    Cell_atoms = pd.DataFrame(columns=['Chain_name', 'Chain_num', 'Residue', 'Type', 'Atom_id', 'x_new', 'y_new', 'z_new'])
    for id_atom in range(Atoms_coord.shape[0]):
        at_coord = Atoms_coord.iloc[id_atom]
        if abs(at_coord['x_new'])<= length_gr and abs(at_coord['y_new'])<= width_gr and abs(at_coord['z_new'])<= high_gr:
            x_ind = at_coord['x_new']+length_gr
            y_ind = at_coord['y_new']+width_gr
            z_ind = at_coord['z_new']+high_gr
            Cell_atoms.loc[len(Cell_atoms.index)] = [at_coord['Chain_name'], at_coord['Chain_num'], at_coord['Residue'], at_coord['Type'], id_atom+1, x_ind, y_ind, z_ind]
    if Cell_atoms.shape[0] < 10:
        with open("Dataset/Diff_struct_new_10.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([str_num, struct_id])

    Cell_atoms_10 = pd.DataFrame(columns=[ 'Index', 'Chain_name', 'Chain_num', 'Residue', 'Type', 'Atom_id', 'x_new', 'y_new', 'z_new'])
    atoms_chain1 = Cell_atoms[Cell_atoms['Chain_num'] == 0]
    atoms_chain2 = Cell_atoms[Cell_atoms['Chain_num'] == 1]
    for i in tqdm(atoms_chain1.index.values):
        atom1 = atoms_chain1.loc[i]
        for j in atoms_chain2.index.values:
            atom2 = atoms_chain2.loc[j]
            b = [atom1['x_new']-atom2['x_new'], atom1['y_new']-atom2['y_new'], atom1['z_new']-atom2['z_new']]
            b_norm = np.linalg.norm(b)
            if b_norm <= 10:
                if i not in Cell_atoms_10['Index'].values:
                    Cell_atoms_10.loc[len(Cell_atoms_10.index)] = [i, atom1['Chain_name'], atom1['Chain_num'], atom1['Residue'], atom1['Type'], atom1['Atom_id'], atom1['x_new'], atom1['y_new'], atom1['z_new']]
                if j not in Cell_atoms_10['Index'].values:
                    Cell_atoms_10.loc[len(Cell_atoms_10.index)] = [j, atom2['Chain_name'], atom2['Chain_num'], atom2['Residue'], atom2['Type'], atom2['Atom_id'], atom2['x_new'], atom2['y_new'], atom2['z_new']]
                continue



    Cell_atoms_10.to_csv(f'Cell_tables_10/{struct_id}_cell_10.csv')



Structure 5th9_90 number 11 in processing...


100%|██████████████████████████████████████████████████████████████████████████████| 1413/1413 [02:16<00:00, 10.37it/s]


Structure 5th9_91 number 12 in processing...


100%|██████████████████████████████████████████████████████████████████████████████| 1408/1408 [02:15<00:00, 10.38it/s]


Structure 5th9_92 number 13 in processing...


...

100%|██████████████████████████████████████████████████████████████████████████████| 1401/1401 [02:20<00:00,  9.97it/s]


Structure 5th9_99 number 20 in processing...


100%|██████████████████████████████████████████████████████████████████████████████| 1390/1390 [02:15<00:00, 10.25it/s]


Structure 5th9_100 number 21 in processing...


100%|██████████████████████████████████████████████████████████████████████████████| 1403/1403 [02:21<00:00,  9.91it/s]


In [ ]:
Cell_atoms_10

,Index,Chain_name,Chain_num,Residue,Type,Atom_id,x_new,y_new,z_new
0,83,A,0,ASP,O,88,25.111622,46.140770,8.459672
1,2515,B,1,ARG,NH2,2888,26.292169,40.160073,15.557036
2,85,A,0,ASP,CG,90,22.395808,48.892172,10.246031
3,2339,B,1,LEU,CD1,2686,22.612862,42.054978,17.271394
4,86,A,0,ASP,OD1,91,23.112244,49.491772,11.074724
...,...,...,...,...,...,...,...,...,...
1105,1277,A,0,PHE,CG,1461,19.759079,54.805993,23.097048
1106,1278,A,0,PHE,CD1,1462,18.887486,54.343097,22.113222
1107,1279,A,0,PHE,CD2,1463,20.689244,55.793701,22.765397
1108,1281,A,0,PHE,CE2,1465,20.752270,56.312180,21.473585
